In [2]:
!pip install ollama


   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 1.5 MB/s eta 0:00:01
   --------------- ------------------------ 0.8/2.0 MB 1.6 MB/s eta 0:00:01
   ------------------------- -------------- 1.3/2.0 MB 1.6 MB/s eta 0:00:01
   ------------------------------- -------- 1.6/2.0 MB 1.6 MB/s eta 0:00:01
   ------------------------------------ --- 1.8/2.0 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 1.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.20.1
    Uninstalling pydantic_core-2.20.1:
      Successfully uninstalled pydantic_core-2.20.1
  Attempting uninstall: 

In [6]:
import ollama

response = ollama.chat(
    model='llama3',
    messages=[
        {'role': 'system', 'content': 'You are a university student giving short feedback on a Machine Learning course.'},
        {'role': 'user', 'content': 'Write a short, polite feedback message about the Machine Learning course.'}
    ]
)

print(response['message']['content'])


Dear Professor,

I wanted to take a moment to express my appreciation for the Machine Learning course this semester. I found the material to be engaging and challenging, and your instruction was clear and supportive throughout.

The assignments were well-structured and helped me to reinforce my understanding of the concepts. I particularly enjoyed the project work, as it allowed me to apply what we learned in a practical way.

One area where I think the course could be improved is in providing more opportunities for students to ask questions and get feedback during lectures. However, overall, I was very pleased with the course and feel that it has given me a strong foundation in Machine Learning.

Thank you again for your hard work and dedication to teaching this course. I'm grateful to have had the opportunity to take it!

Best regards,
[Your Name]
